In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

import findspark
findspark.init()

# Import and start a Spark session.
from pyspark.sql import *
import pyspark.sql.functions as F

spark = SparkSession.builder.getOrCreate()

/home/pierre/.local/lib/python3.7/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


In [2]:
def load_insights(months):
    return {month: spark.read.parquet('hashtags_insights_{}.parquet'.format(month)) for month in months} 

We start by loading the different hashtags insights for each months. The month of January is omitted as it is corrupted, the month of June is also omitted as it is too small (we only have data for an extract of two days), and the month december does not exists.

In [35]:
hashtags = load_insights(['02', '03', '04', '05', '07', '08', '09', '10', '11'])

In [36]:
hashtags["03"].columns

['tag',
 'count',
 'print',
 'print_1',
 'print_2',
 'print_3',
 'print_4',
 'print_5',
 'print_6',
 'print_7',
 'print_8',
 'print_9',
 'print_10',
 'print_11',
 'print_12',
 'print_13',
 'print_14',
 'print_15',
 'print_16',
 'print_17',
 'print_18',
 'print_19',
 'print_20',
 'print_21',
 'print_22',
 'print_23',
 'print_24',
 'print_25',
 'print_26',
 'print_27',
 'print_28',
 'print_29',
 'print_30',
 'print_31',
 'count_1',
 'count_2',
 'count_3',
 'count_4',
 'count_5',
 'count_6',
 'count_7',
 'count_8',
 'count_9',
 'count_10',
 'count_11',
 'count_12',
 'count_13',
 'count_14',
 'count_15',
 'count_16',
 'count_17',
 'count_18',
 'count_19',
 'count_20',
 'count_21',
 'count_22',
 'count_23',
 'count_24',
 'count_25',
 'count_26',
 'count_27',
 'count_28',
 'count_29',
 'count_30',
 'count_31']

In [34]:
all_days = [str(i) for i in range(1, 32)]
all_months = [str(i).zfill(2) for i in range(1, 13)]

In [21]:
allColumns = \
    ["tag", "count", "print"] + \
    [month + "_print_" + day for day in all_days for month in all_months] + \
    [month + "_count_" + day for day in all_days for month in all_months]

We define a function which will allow us to unify the columns of every sub dataframes by renaming some columns and adding some others.

In [22]:
def rename_columns(insights, prefix):
    printCols = [c for c in insights.columns if "print_" in c]
    nameCols = [c for c in insights.columns if "count_" in c]
    for x in printCols + nameCols:
        insights = insights.withColumnRenamed(x, prefix + "_" + x)
    for x in allColumns:
        if not x in insights.columns:
            insights = insights.withColumn(x, F.lit(0))
    return insights

We then apply this function to all of our insights.

In [7]:
insights = [rename_columns(hashtags[key], key) for key in hashtags]

In [41]:
insights[1].where(F.col('03_count_1') != 0).count()

279535

We merge every insights in one unique dataframe.

In [8]:
from functools import reduce

def unionAll(dfs):
    return reduce(DataFrame.unionAll, dfs)
unifiedInsights = unionAll(insights)

As some hashtags might appear in different months, we need to merge duplicates occurencies of the same hashtags.

In [9]:
columnsToSum = [x for x in unifiedInsights.columns if x != "tag"]
aggExpressions = [F.sum(F.col(col)).alias(col) for col in columnsToSum]

In [10]:
unifiedInsightsGB = unifiedInsights.groupBy('tag').agg(*aggExpressions)

In [12]:
unifiedInsightsGB.write.mode("overwrite").parquet("unifiedinsights.parquet")

In [14]:
unifiedInsights = spark.read.parquet("unifiedinsights.parquet")

In [25]:
for x in allColumns:
    if unifiedInsights.where(F.col(x) != 0).count() == 0:
        unifiedInsights = unifiedInsights.drop(x)

AnalysisException: "cannot resolve '`01_count_01`' given input columns: [02_print_20, 02_print_18, 02_count_5, print, 02_print_4, 02_print_14, 02_print_22, 02_print_10, 02_count_15, 02_count_7, 02_count_10, 02_count_25, 02_print_12, 02_print_19, 02_print_6, 02_count_2, 02_print_31, 02_print_27, 02_print_17, 02_print_21, 02_count_27, 02_count_30, 02_count_21, 02_print_9, 02_count_19, 02_count_18, 02_count_1, count, 02_print_1, 02_count_29, 02_count_11, 02_print_28, 02_print_15, 02_print_3, 02_count_6, 02_print_30, 02_count_9, 02_print_13, 02_count_13, 02_print_29, 02_count_12, 02_print_5, 02_print_11, 02_print_26, 02_count_26, 02_count_24, 02_count_31, 02_print_24, 02_print_23, 02_count_4, 02_print_7, 02_count_14, 02_print_8, 02_count_8, 02_count_3, 02_count_22, 02_print_16, 02_print_2, 02_count_23, 02_count_17, 02_count_28, 02_count_20, 02_count_16, 02_print_25];;\n'Filter NOT ('01_count_01 = 0)\n+- AnalysisBarrier\n      +- Project [count#806190L, print#806191L, 02_print_1#806192L, 02

In [ ]:
[x for x in unifiedInsights.columns if "01" in x]

In [ ]:
len(unifiedInsights.columns)

In [20]:
[x for x in unifiedInsights.columns if "01" in x]

['01_print_01',
 '02_print_01',
 '03_print_01',
 '04_print_01',
 '05_print_01',
 '06_print_01',
 '07_print_01',
 '08_print_01',
 '09_print_01',
 '10_print_01',
 '11_print_01',
 '12_print_01',
 '01_print_02',
 '01_print_03',
 '01_print_04',
 '01_print_05',
 '01_print_06',
 '01_print_07',
 '01_print_08',
 '01_print_09',
 '01_print_10',
 '01_print_11',
 '01_print_12',
 '01_print_13',
 '01_print_14',
 '01_print_15',
 '01_print_16',
 '01_print_17',
 '01_print_18',
 '01_print_19',
 '01_print_20',
 '01_print_21',
 '01_print_22',
 '01_print_23',
 '01_print_24',
 '01_print_25',
 '01_print_26',
 '01_print_27',
 '01_print_28',
 '01_print_29',
 '01_print_30',
 '01_print_31']

In [ ]:
unifiedInsights.write.mode("overwrite").parquet("insights.parquet")

In [33]:
unifiedInsights.where(F.col('03_count_1') != 0).count()


AnalysisException: "cannot resolve '`03_count_1`' given input columns: [02_print_20, 02_print_18, 02_count_5, print, 02_print_4, 02_print_14, 02_print_22, 02_print_10, 02_count_15, 02_count_7, 02_count_10, 02_count_25, 02_print_12, 02_print_19, 02_print_6, 02_count_2, 02_print_31, 02_print_27, 02_print_17, 02_print_21, 02_count_27, 02_count_30, 02_count_21, 02_print_9, 02_count_19, 02_count_18, 02_count_1, count, 02_print_1, 02_count_29, 02_count_11, 02_print_28, 02_print_15, 02_print_3, 02_count_6, 02_print_30, 02_count_9, 02_print_13, 02_count_13, 02_print_29, 02_count_12, 02_print_5, 02_print_11, 02_print_26, 02_count_26, 02_count_24, 02_count_31, 02_print_24, 02_print_23, 02_count_4, 02_print_7, 02_count_14, 02_print_8, 02_count_8, 02_count_3, 02_count_22, 02_print_16, 02_print_2, 02_count_23, 02_count_17, 02_count_28, 02_count_20, 02_count_16, 02_print_25];;\n'Filter NOT ('03_count_1 = 0)\n+- AnalysisBarrier\n      +- Project [count#806190L, print#806191L, 02_print_1#806192L, 02_p

In [32]:
hashtags["03"].columns

['tag',
 'count',
 'print',
 'print_1',
 'print_2',
 'print_3',
 'print_4',
 'print_5',
 'print_6',
 'print_7',
 'print_8',
 'print_9',
 'print_10',
 'print_11',
 'print_12',
 'print_13',
 'print_14',
 'print_15',
 'print_16',
 'print_17',
 'print_18',
 'print_19',
 'print_20',
 'print_21',
 'print_22',
 'print_23',
 'print_24',
 'print_25',
 'print_26',
 'print_27',
 'print_28',
 'print_29',
 'print_30',
 'print_31',
 'count_1',
 'count_2',
 'count_3',
 'count_4',
 'count_5',
 'count_6',
 'count_7',
 'count_8',
 'count_9',
 'count_10',
 'count_11',
 'count_12',
 'count_13',
 'count_14',
 'count_15',
 'count_16',
 'count_17',
 'count_18',
 'count_19',
 'count_20',
 'count_21',
 'count_22',
 'count_23',
 'count_24',
 'count_25',
 'count_26',
 'count_27',
 'count_28',
 'count_29',
 'count_30',
 'count_31']